In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory



# Any results you write to the current directory are saved as output.


# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# 
base_path = "input/"
train=pd.read_csv(base_path + 'train.csv')
test=pd.read_csv(base_path + 'test.csv')
#more about kinetic features  developed  by Daia Alexandru    here  on the next  blog  please  read  last article :
#https://alexandrudaia.quora.com/

##############################################creatinng   kinetic features for  train #####################################################
def  kinetic(row):
    probs=np.unique(row,return_counts=True)[1]/len(row)
    kinetic=np.sum(probs**2)
    return kinetic
    

first_kin_names=[col for  col in train.columns  if '_ind_' in col]
subset_ind=train[first_kin_names]
kinetic_1=[]
for row in range(subset_ind.shape[0]):
    row=subset_ind.iloc[row]
    k=kinetic(row)
    kinetic_1.append(k)
second_kin_names= [col for  col in train.columns  if '_car_' in col and col.endswith('cat')]
subset_ind=train[second_kin_names]
kinetic_2=[]
for row in range(subset_ind.shape[0]):
    row=subset_ind.iloc[row]
    k=kinetic(row)
    kinetic_2.append(k)
third_kin_names= [col for  col in train.columns  if '_calc_' in col and  not col.endswith('bin')]
subset_ind=train[second_kin_names]
kinetic_3=[]
for row in range(subset_ind.shape[0]):
    row=subset_ind.iloc[row]
    k=kinetic(row)
    kinetic_3.append(k)
fd_kin_names= [col for  col in train.columns  if '_calc_' in col and  col.endswith('bin')]
subset_ind=train[fd_kin_names]
kinetic_4=[]
for row in range(subset_ind.shape[0]):
    row=subset_ind.iloc[row]
    k=kinetic(row)
    kinetic_4.append(k)
train['kinetic_1']=np.array(kinetic_1)
train['kinetic_2']=np.array(kinetic_2)
train['kinetic_3']=np.array(kinetic_3)
train['kinetic_4']=np.array(kinetic_4)

############################################reatinng   kinetic features for  test###############################################################

first_kin_names=[col for  col in test.columns  if '_ind_' in col]
subset_ind=test[first_kin_names]
kinetic_1=[]
for row in range(subset_ind.shape[0]):
    row=subset_ind.iloc[row]
    k=kinetic(row)
    kinetic_1.append(k)
second_kin_names= [col for  col in test.columns  if '_car_' in col and col.endswith('cat')]
subset_ind=test[second_kin_names]
kinetic_2=[]
for row in range(subset_ind.shape[0]):
    row=subset_ind.iloc[row]
    k=kinetic(row)
    kinetic_2.append(k)
third_kin_names= [col for  col in test.columns  if '_calc_' in col and  not col.endswith('bin')]
subset_ind=test[second_kin_names]
kinetic_3=[]
for row in range(subset_ind.shape[0]):
    row=subset_ind.iloc[row]
    k=kinetic(row)
    kinetic_3.append(k)
fd_kin_names= [col for  col in test.columns  if '_calc_' in col and  col.endswith('bin')]
subset_ind=test[fd_kin_names]
kinetic_4=[]
for row in range(subset_ind.shape[0]):
    row=subset_ind.iloc[row]
    k=kinetic(row)
    kinetic_4.append(k)
test['kinetic_1']=np.array(kinetic_1)
test['kinetic_2']=np.array(kinetic_2)
test['kinetic_3']=np.array(kinetic_3)
test['kinetic_4']=np.array(kinetic_4)

##################################################################end  of kinetics ############################################################################
from sklearn import *
import xgboost as xgb
from multiprocessing import *
import numpy as np
import pandas as pd
from sklearn import *
import xgboost as xgb
import lightgbm as lgb
from multiprocessing import *


d_median = train.median(axis=0)
d_mean = train.mean(axis=0)

def transform_df(df):
    df = pd.DataFrame(df)
    dcol = [c for c in df.columns if c not in ['id','target']]
    df['ps_car_13_x_ps_reg_03'] = df['ps_car_13'] * df['ps_reg_03']
    df['negative_one_vals'] = np.sum((df[dcol]==-1).values, axis=1)
    for c in dcol:
        if '_bin' not in c:
            df[c+str('_median_range')] = (df[c].values > d_median[c]).astype(int)
            df[c+str('_mean_range')] = (df[c].values > d_mean[c]).astype(int)
    return df

def multi_transform(df):
    print('Init Shape: ', df.shape)
    p = Pool(cpu_count())
    df = p.map(transform_df, np.array_split(df, cpu_count()))
    df = pd.concat(df, axis=0, ignore_index=True).reset_index(drop=True)
    p.close(); p.join()
    print('After Shape: ', df.shape)
    return df

def gini(y, pred):
    g = np.asarray(np.c_[y, pred, np.arange(len(y)) ], dtype=np.float)
    g = g[np.lexsort((g[:,2], -1*g[:,1]))]
    gs = g[:,0].cumsum().sum() / g[:,0].sum()
    gs -= (len(y) + 1) / 2.
    return gs / len(y)

def gini_xgb(pred, y):
    #y = y.get_label()
    return 'gini', gini(y, pred) / gini(y, y)

params = {'eta': 0.02, 'max_depth': 4, 'subsample': 0.9, 'colsample_bytree': 0.9, 'objective': 'binary:logistic', 'eval_metric': 'auc', 'seed': 99, 'silent': True}
x1, x2, y1, y2 = model_selection.train_test_split(train, train['target'], test_size=0.25, random_state=99)

C:\Users\Crossbell\Anaconda2\envs\tensorflow\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\Users\Crossbell\Anaconda2\envs\tensorflow\lib\site-packages\sklearn\grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)
C:\Users\Crossbell\Anaconda2\envs\tensorflow\lib\site-packages\sklearn\learning_curve.py:22: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the functions are moved.

In [2]:
x1 = transform_df(x1)
y1 = x1['target']
x2 = transform_df(x2)
y2 = x2['target']
test = transform_df(test)

col = [c for c in x1.columns if c not in ['id','target']]
x1 = x1[col]
x2 = x2[col]

watchlist = [(xgb.DMatrix(x1, y1), 'train'), (xgb.DMatrix(x2, y2), 'valid')]
model = xgb.train(params, xgb.DMatrix(x1, y1), 5000,  watchlist, verbose_eval=50, early_stopping_rounds=100)
print(gini_xgb(model.predict(xgb.DMatrix(x2), ntree_limit=model.best_ntree_limit), y2))
test['target'] = model.predict(xgb.DMatrix(test[col]), ntree_limit=model.best_ntree_limit)
test[['id','target']].to_csv(base_path + 'test_uberKinetics.csv', index=False, float_format='%.5f')

train = transform_df(train)
train['target'] = model.predict(xgb.DMatrix(train[col]), ntree_limit=model.best_ntree_limit)
train[['id','target']].to_csv(base_path + 'train_uberKinetics.csv', index=False, float_format='%.5f')

Will train until valid error hasn't decreased in 100 rounds.
[0]	train-auc:0.593750	valid-auc:0.591457
[50]	train-auc:0.621774	valid-auc:0.618708
[100]	train-auc:0.625400	valid-auc:0.621529
[150]	train-auc:0.632034	valid-auc:0.624779
[200]	train-auc:0.638926	valid-auc:0.628299
[250]	train-auc:0.644247	valid-auc:0.630823
[300]	train-auc:0.649415	valid-auc:0.633325
[350]	train-auc:0.653584	valid-auc:0.634883
[400]	train-auc:0.657280	valid-auc:0.636280
[450]	train-auc:0.660489	valid-auc:0.637117
[500]	train-auc:0.663309	valid-auc:0.637756
[550]	train-auc:0.665797	valid-auc:0.638059
[600]	train-auc:0.668535	valid-auc:0.638452
[650]	train-auc:0.671038	valid-auc:0.638729
[700]	train-auc:0.673369	valid-auc:0.638719
[750]	train-auc:0.675651	valid-auc:0.638896
[800]	train-auc:0.677828	valid-auc:0.639183
[850]	train-auc:0.679838	valid-auc:0.639194
[900]	train-auc:0.681754	valid-auc:0.639246
[950]	train-auc:0.683865	valid-auc:0.639375
[1000]	train-auc:0.685654	valid-auc:0.639368
[1050]	train-auc: